In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/parkinsons-drawings/spiral/training/healthy/V06HE03.png
/kaggle/input/parkinsons-drawings/spiral/training/healthy/V55HE09.png
/kaggle/input/parkinsons-drawings/spiral/training/healthy/V55HE11.png
/kaggle/input/parkinsons-drawings/spiral/training/healthy/V09HE03.png
/kaggle/input/parkinsons-drawings/spiral/training/healthy/V08HE03.png
/kaggle/input/parkinsons-drawings/spiral/training/healthy/V07HE03.png
/kaggle/input/parkinsons-drawings/spiral/training/healthy/V12HE03.png
/kaggle/input/parkinsons-drawings/spiral/training/healthy/V04HE03.png
/kaggle/input/parkinsons-drawings/spiral/training/healthy/V11HE03.png
/kaggle/input/parkinsons-drawings/spiral/training/healthy/V05HE02.png
/kaggle/input/parkinsons-drawings/spiral/training/healthy/V12HE01.png
/kaggle/input/parkinsons-drawings/spiral/training/healthy/V55HE05.png
/kaggle/input/parkinsons-drawings/spiral/training/healthy/V02HE03.png
/kaggle/input/parkinsons-drawings/spiral/training/healthy/V55HE03.png
/kaggle/input/parkin

In [2]:
!tree -d /kaggle/input/parkinsons-drawings/

/kaggle/input/parkinsons-drawings/
├── drawings
│   ├── spiral
│   │   ├── testing
│   │   │   ├── healthy
│   │   │   └── parkinson
│   │   └── training
│   │       ├── healthy
│   │       └── parkinson
│   └── wave
│       ├── testing
│       │   ├── healthy
│       │   └── parkinson
│       └── training
│           ├── healthy
│           └── parkinson
├── spiral
│   ├── testing
│   │   ├── healthy
│   │   └── parkinson
│   └── training
│       ├── healthy
│       └── parkinson
└── wave
    ├── testing
    │   ├── healthy
    │   └── parkinson
    └── training
        ├── healthy
        └── parkinson

29 directories


In [3]:
import os
import numpy as np
import pandas as pd
from keras.optimizers import Adadelta
from keras.layers import *
from keras import backend as K
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.models import Model,load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau,EarlyStopping

#from scipy.misc import toimage,imresize
from skimage import exposure
from PIL import Image
import cv2

In [4]:
spirals_train_folder = '../input/parkinsons-drawings/spiral/training/'
spirals_val_folder = '../input/parkinsons-drawings/spiral/testing/'
waves_train_folder = '../input/parkinsons-drawings/wave/training/'
waves_val_folder = '../input/parkinsons-drawings/wave/testing/'

batch_size = 24

# histogram equalizer
def eqz_plz(img):
    return exposure.equalize_hist(img)


spiral_datagen = ImageDataGenerator(rotation_range=360, # they're spirals.
                                    width_shift_range=0.1,
                                    height_shift_range=0.1,
                                    brightness_range=(0.5,1.5),
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    preprocessing_function=eqz_plz,
                                    vertical_flip=True)

wave_datagen = ImageDataGenerator(rotation_range=5,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  brightness_range=(0.5,1.5),
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  preprocessing_function=eqz_plz,
                                  vertical_flip=True)


spiral_train_generator = spiral_datagen.flow_from_directory(directory=os.path.abspath(spirals_train_folder),
                                                            target_size=(256, 256),
                                                            color_mode="grayscale",
                                                            batch_size=batch_size,
                                                            class_mode="binary",
                                                            shuffle=True,
                                                            seed=666)

spiral_val_generator = spiral_datagen.flow_from_directory(directory=os.path.abspath(spirals_val_folder),
                                                            target_size=(256, 256),
                                                            color_mode="grayscale",
                                                            batch_size=batch_size,
                                                            class_mode="binary",
                                                            shuffle=True,
                                                            seed=710)

wave_train_generator = wave_datagen.flow_from_directory(directory=os.path.abspath(waves_train_folder),
                                                        target_size=(256, 512), # HxW in machine learning, WxH in computer vision
                                                        color_mode="grayscale",
                                                        batch_size=batch_size,
                                                        class_mode="binary",
                                                        shuffle=True,
                                                        seed=420)

wave_val_generator = wave_datagen.flow_from_directory(directory=os.path.abspath(waves_val_folder),
                                                        target_size=(256, 512),
                                                        color_mode="grayscale",
                                                        batch_size=batch_size,
                                                        class_mode="binary",
                                                        shuffle=True,
                                                        seed=420)

Found 72 images belonging to 2 classes.
Found 30 images belonging to 2 classes.
Found 72 images belonging to 2 classes.
Found 30 images belonging to 2 classes.


In [5]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss',patience=12,min_lr=1e-9,verbose=1)
early_stop = EarlyStopping(monitor='val_loss',patience=16,verbose=1)

In [6]:
K.clear_session()

def custom_model(mode):
    if (mode == 'spirals') or (mode == 'spiral'):
        input_layer = Input(shape=(256,256,1),name=f'{mode}_input_layer')
    elif (mode == 'waves') or (mode == 'wave'):
        input_layer = Input(shape=(256,512,1),name=f'{mode}_input_layer')

    m1 = Conv2D(256,(5,5),dilation_rate=4,kernel_initializer='glorot_normal',kernel_regularizer=l2(0.001),activation='relu',padding='same')(input_layer)
    p1 = MaxPool2D((9,9),strides=2)(m1)
    m2 = Conv2D(128,(5,5),dilation_rate=2,kernel_initializer='glorot_normal',kernel_regularizer=l2(0.001),activation='relu',padding='same')(p1)
    p2 = MaxPool2D((7,7),strides=2)(m2)
    m3 = Conv2D(64,(3,3),kernel_initializer='glorot_normal',kernel_regularizer=l2(0.001),activation='relu',padding='same')(p2)
    p3 = MaxPool2D((5,5),strides=2)(m3)
    f1 = Flatten()(p3)
    d1 = Dense(666,activation='relu')(f1)
    d2 = Dense(1,activation='softmax')(d1)
    
    this_model = Model(input_layer,d2)
    #this_model.summary()
    return this_model

In [7]:
spiral_model = custom_model(mode='spirals') # early stopping epoch 89: val_loss 0.4796, val_acc 0.8274
spiral_model.compile(optimizer=Adadelta(learning_rate=3.15e-5), loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
spiral_model.fit_generator(spiral_train_generator,
                           validation_data=spiral_val_generator,
                           epochs=666,
                           steps_per_epoch=(2000//batch_size),
                           validation_steps=(800//batch_size),
                           verbose=1)

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/opt/conda/lib/python3.7/site-packages/skimage/exposure/exposure.py:181: UserWarning: This might be a color image. The histogram will be computed on the flattened image. You can instead apply this function to each color channel.
  hist, bin_centers = histogram(image, nbins)


Epoch 1/666
83/83 [==============================] - 7s 30ms/step - loss: 0.9509 - accuracy: 0.4990 - val_loss: 0.9494 - val_accuracy: 0.5000


In [9]:
spiral_model.save('../output/custom_model_spirals.h5')

In [10]:
waves_model = custom_model(mode='waves')
waves_model.compile(optimizer=Adam(lr=3.15e-5),loss='binary_crossentropy',metrics=['accuracy'])

In [11]:
waves_model.fit_generator(wave_train_generator,
                          validation_data=wave_val_generator,
                          epochs=666,
                          steps_per_epoch=(2000//batch_size),
                          validation_steps=(800//batch_size),
                          
                          verbose=1)

Epoch 1/666
83/83 [==============================] - 7s 44ms/step - loss: 0.9917 - accuracy: 0.5015 - val_loss: 0.9567 - val_accuracy: 0.5000


In [12]:


waves_model.save('../output/custom_model_waves.h5')



In [13]:
doc_input = concatenate([spiral_model.output,waves_model.output])
dense_doc_1 = Dense(69,activation='relu')(doc_input)
dense_doc_2 = Dense(1,activation='sigmoid')(dense_doc_1)

In [14]:
def multiple_generators(gen1,gen2):
    while True:
        X1 = gen1.next()
        X2 = gen2.next()
        yield [X1[0], X2[0]], ((X1[1]+X2[1])/2)
            
input_generator = multiple_generators(spiral_train_generator,wave_train_generator)       
test_generator = multiple_generators(spiral_train_generator,wave_train_generator)      

def disable_trainable(model):
    for layer in model.layers:
        layer.trainable = False
        
disable_trainable(spiral_model)
disable_trainable(waves_model)
spiral_model.compile(optimizer=Adadelta(lr=5.11089622e-5), loss='binary_crossentropy', metrics=['accuracy'])
waves_model.compile(optimizer=Adadelta(lr=5.11089622e-5), loss='binary_crossentropy', metrics=['accuracy'])

doctor_model = Model(inputs=[spiral_model.input,waves_model.input],outputs=dense_doc_2)
doctor_model.compile(optimizer=Adadelta(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:


reduce_lr_2 = ReduceLROnPlateau(monitor='val_loss',patience=8,rate=0.4,min_lr=1e-12,verbose=1)
early_stop_2 = EarlyStopping(monitor='val_loss',patience=24,verbose=1)

doctor_model.fit_generator(input_generator,
                           validation_data=test_generator,
                           epochs=666,
                           steps_per_epoch=(2000//batch_size),
                           validation_steps=(800//batch_size),
                           callbacks=[reduce_lr_2,early_stop_2],verbose=1)



Epoch 1/666
67/83 [=======================>......] - ETA: 13s - loss: 1.2142 - accuracy: 0.2461

In [ ]:
doctor_model.save('../output/custom_model_doctor.h5')